In [1]:
import pandas as pd
from src.Navigator import Navigator
from src.EventReader import EventReader
from src.SQLIO import SQLIO
from src.EventWriter import EventWriter
from multiprocessing import Process
import threading
import time

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
class MyThread(threading.Thread):
    def __init__(self, name, page_nr, event_nr):
        super().__init__()
        self.page_nr = page_nr
        self.event_nr = event_nr
        self.name = name
        
    def run(self):
        print("{} started!".format(self.name) )
        try:
            self.crawl_event(self.page_nr, self.event_nr)  
        except:
            print("{} failed!".format(self.name))
            raise
        print("{} finished!".format(self.name))
        
    def crawl_event(self, page_nr, event_nr):
        n = Navigator(True)
        try:

            n.open_search()
            n.go_to_page(page_nr)
            n.open_event(event_nr)
            e = EventReader(n)
            event = e.read()
            EventWriter.write(event)
        finally:
            n.quit()
              
              
if __name__ == '__main__':
    threads = []
    for page_nr in range(1,3):
        for event_nr in range(10):
            print(threading.active_count())
            while threading.active_count() < 20:
                time.sleep(0.1)
            
            mythread = MyThread(name = "Thread-(p={},e={})".format(page_nr,event_nr), page_nr = page_nr, event_nr = event_nr) 
            mythread.start()
            threads.append(mythread)
        
    for thread in threads:
        thread.join()
    print('done')

5


KeyboardInterrupt: 